# Advanced Python - Building Scalable Applications

### Module 4

#### Synchronization mechanisms for threads and processes
 - Synchronizing flow-control using ```Barrier```
 - Mutual exclusion patterns using ```Lock``` and ```RLock```
 - Wait/notify patterns using ```Condition``` and ```Event```
 - Bandwidth/Resource management and control using ```Semaphore``` and ```BoundedSemaphore```
 - Producer/Consumer patterns using ```Queue```
 - Using ```threading.local()``` to manage thread-local data

#### Sharing and Exchanging data between processes (Overview)
 - Streaming data using ```Pipe``` and ```Queue```
 - Sharing counters and buffers using ```Value``` and ```Array```
 - Sharing python lists and dictionaries using ```Manager```
 - Creating and managing shared memory using ```multiprocessing.shared_memory``` features
